# SQL ПРоверка данных  читательского сервиса

**Легенда**

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Описание данных**

*Таблица `books`*

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

*Таблица `authors`*

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

*Таблица `publishers`*

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

*Таблица `ratings`*

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

*Таблица `reviews`*

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.


**Задачи:**

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так получиться исключить из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;
- Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.


**Загрузка библиотек**

In [1]:
import pandas as pd
import seaborn as sns
import warnings
import numpy as np
from sqlalchemy import create_engine 

sns.set_style('darkgrid')
warnings.filterwarnings("ignore")
pd.set_option('chained_assignment', None)
np.random.seed(42)

**Oглавление:**
<a id="Contents"></a> 

1. Подключение к БД
2. [Знакомство с данными](#Start)
3. [Анализ данных](#Analysis)
4. [Итоги](#Presentation) 

## Подключение к БД

In [2]:
db_config = {'user': 'praktikum_student', 
            'pwd': 'Sdf4$2;d-d30pp', 
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, 
            'db': 'data-analyst-final-project-db'} 


connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string) 


def select_sql(sql):
    """вспомогательная функция для запросов"""
    return pd.io.sql.read_sql(sql, con = engine)

## Знакомство с данными
<a id="Start"></a> 

**Задачи:**
 - [x] Загрузить данные.
 - [x] Изучить общую информацию о датасетах.

### Таблица книги

In [3]:
sql = '''SELECT *
FROM   books t
LIMIT  10 '''

select_sql(sql)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


### Таблица авторы

In [4]:
sql = '''SELECT *
FROM   authors t
LIMIT  10 '''

select_sql(sql)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts


### Таблица издатели

In [5]:
sql = '''SELECT *
FROM   publishers t
LIMIT  10 '''

select_sql(sql)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
5,6,Aladdin
6,7,Aladdin Paperbacks
7,8,Albin Michel
8,9,Alfred A. Knopf
9,10,Alfred A. Knopf Books for Young Readers


### Таблица рейтинги

In [6]:
sql = '''SELECT *
FROM   ratings t
LIMIT  10 '''

select_sql(sql)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


### Таблица обзоры

In [7]:
sql = '''SELECT *
FROM   reviews t
LIMIT  10 '''

select_sql(sql)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


In [8]:
# ячейка ревьюера: можно подсчитать количество строк сразу с использованием небольшой оконной функции:

pd.io.sql.read_sql('''
SELECT *,
       COUNT(*) OVER() AS table_size
FROM reviews 
LIMIT 1        
'''
, con = engine)

,review_id,book_id,username,text,table_size
0,1,1,brandtandrea,Mention society tell send professor analysis. ...,2793


**Вывод:**
    
    В БД хранится 5 таблиц с базовыми данными, визуально данные соответствуют заявленым, редактирование не требуется

## Анализ данных

<a id="Analysis"></a> 
[к оглавлению](#Contents)
    
**Задачи:**

- [x] Посчитать, сколько книг вышло после 1 января 2000 года;
- [x] Для каждой книги посчитать количество обзоров и среднюю оценку;
- [x] Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так получиться исключить из анализа брошюры;
- [x] Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;
- [x] Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

### Колличество книг, вышедших после 1.01.2000

In [9]:
sql = '''
SELECT 
    COUNT(DISTINCT(book_id)) as quantity
FROM 
    books t
WHERE 
    t.publication_date > '2000-01-01'
'''
select_sql(sql)

,quantity
0,819


*Вывод:*
После 1.01.2000 было выпущено 819 книг

### Количество обзоров и средняя оценка для книг

In [10]:
sql = '''
WITH review AS
(
         SELECT   count (DISTINCT(review_id)) AS review_cnt,
                  book_id
         FROM     reviews
         GROUP BY book_id), rating AS
(
         SELECT   avg(rating :: real) AS avg_rating,
                  book_id
         FROM     ratings
         GROUP BY book_id)
SELECT    t.book_id,
          t.title,
          CASE
                    WHEN r.review_cnt IS NULL THEN 0
                    ELSE r.review_cnt
          END AS review_cnt,
          CASE
                    WHEN ra.avg_rating::real IS NULL THEN 0
                    ELSE ra.avg_rating::real
          END AS rating
FROM      books t
LEFT JOIN review r
ON        t.book_id = r.book_id
LEFT JOIN rating ra
ON        r.book_id = ra.book_id
ORDER BY  review_cnt DESC limit 15'''

select_sql(sql)

,book_id,title,review_cnt,rating
0,948,Twilight (Twilight #1),7,3.66250
1,854,The Road,6,3.77273
2,207,Eat Pray Love,6,3.39583
3,656,The Book Thief,6,4.26415
4,733,The Giver (The Giver #1),6,3.75000
5,779,The Lightning Thief (Percy Jackson and the Oly...,6,4.08065
6,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41463
7,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.28750
8,497,Outlander (Outlander #1),6,4.12500
9,627,The Alchemist,6,3.78947


In [11]:
sql = '''
WITH review AS
(
         SELECT   Count (DISTINCT(review_id)) AS review_cnt,
                  book_id
         FROM     reviews
         GROUP BY book_id), rating AS
(
         SELECT   Avg(rating :: real) AS avg_rating,
                  book_id
         FROM     ratings
         GROUP BY book_id)
SELECT    t.book_id,
          t.title,
          CASE
                    WHEN r.review_cnt IS NULL THEN 0
                    ELSE r.review_cnt
          END AS review_cnt,
          CASE
                    WHEN ra.avg_rating::real IS NULL THEN 0
                    ELSE ra.avg_rating::real
          END AS rating
FROM      books t
LEFT JOIN review r
ON        t.book_id = r.book_id
LEFT JOIN rating ra
ON        r.book_id = ra.book_id
ORDER BY  rating  DESC  limit 15'''

select_sql(sql)

,book_id,title,review_cnt,rating
0,182,Dead Souls,2,5.0
1,275,Geek Love,2,5.0
2,169,Crucial Conversations: Tools for Talking When ...,2,5.0
3,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.0
4,229,Evil Under the Sun (Hercule Poirot #24),2,5.0
5,224,Evening Class,2,5.0
6,57,Act of Treason (Mitch Rapp #9),2,5.0
7,20,A Fistful of Charms (The Hollows #4),2,5.0
8,55,A Woman of Substance (Emma Harte Saga #1),2,5.0
9,76,Angels Fall,2,5.0


*Вывод:*

- по количеству отзывов ледируют Сумерки, но в топ 15 вошли и произведения Роулинг, Толкиена, т.е преобладает фентези классика
- у книг с высокой оценкой мало отзывов 
- среди лидеров по оценке встречается Российская класска, например Мёртвые души.

### Издательство с наибольшим количеством книг, толще 50 стр

In [12]:
sql = '''
WITH books_50 AS
(
         SELECT   count(DISTINCT(book_id)) AS book_cnt,
                  t.publisher_id
         FROM     books t
         WHERE    t.num_pages > 50
         GROUP BY publisher_id)
SELECT    p.publisher as best_publisher,
          CASE
                    WHEN b.book_cnt IS NULL THEN 0
                    ELSE b.book_cnt
          END AS books_cnt
FROM      publishers p
LEFT JOIN books_50 b
ON        p.publisher_id = b.publisher_id
ORDER BY  books_cnt DESC limit 1
'''

select_sql(sql)

,best_publisher,books_cnt
0,Penguin Books,42


*Вывод:*
Больше всего книг выпустило издательство Penguin Books	

### Автор книги с самой высокой оценкой


Будем брать только книги более 50 страниц

In [13]:
sql = '''
WITH rating_50 AS
(
         SELECT   Count(rating_id)     AS cnt,
                  Avg(rating::real) AS avg_rating,
                  book_id
         FROM     ratings
         GROUP BY book_id
         HAVING   Count(rating_id) >= 50)
SELECT    a.author,
          AVG(r.avg_rating::real) as rating
FROM      books t
INNER JOIN rating_50 r
ON        r.book_id = t.book_id
INNER JOIN authors a
ON        a.author_id = t.author_id

GROUP BY
    a.author_id,
    a.author
ORDER BY rating DESC
LIMIT 1
'''

select_sql(sql)

,author,rating
0,J.K. Rowling/Mary GrandPré,4.283844


*Вывод:*
Самая высокая оценка у книг Роулинг о Гарри Поттере

### Среднее количество обзоров от пользователей

Будем брать пользователей, которые поставили больше 50 оценок.

In [15]:
sql ='''
WITH selected_50
     AS (SELECT username,
                count(*) AS reviews_num
         FROM   reviews
         WHERE  username IN (SELECT username
                             FROM   ratings
                             GROUP  BY username
                             HAVING Count(rating_id) > 50)
         GROUP  BY username)
SELECT ROUND(avg(reviews_num), 2)
FROM   selected_50 

'''
select_sql(sql)

,round
0,24.33


*Вывод:*
Пользователи поставившие более 50 оценок, в среднем написали 24 отзыва.


### Итоги
<a id="Presentation"></a> 
[к оглавлению](#Contents)

Проанализировав имеющиеся данные мы получили следующие результаты:
 - После 1.01.2000 было выпущено 819 книг
 - по количеству отзывов ледируют Сумерки, но в топ 15 вошли и произведения Роулинг, Толкиена, т.е преобладает фентези классика
 - у книг с высокой оценкой мало отзывов
 - среди лидеров по оценке встречается Российская класска, например Мёртвые души.
 - Больше всего книг выпустило издательство Penguin Books, это 42 книги
 - Самая высокая оценка у книг Роулинг о Гарри Поттере - 4
 - Пользователи поставившие более 50 оценок, в среднем написали 24 отзыва.
 
